# Resize Selected Images

Remove images smaller than 256x256 pixels

Center-crop images and/or potentially downsample very large images

Save to new folders and remake Excel file of accompanying data

In [2]:
# Packages
import pandas as pd
import os
import shutil
import time

# Importing the image processing functions
from Image_Processing import *

In [3]:
# Load in selected cars annos
selected_cars_annos = pd.read_excel("~/Box/INFO 290T Project/Intermediate Data/selected_cars_annos.xlsx")
selected_cars_annos

,Class,x1,y1,x2,y2,old_class,old_class_name,width,height,num_pixels,test_80_20,new_file_path,new_file_name
0,SUV,111,54,365,190,1,AM General Hummer SUV 2000,373,216,80568,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_00684.jpg
1,SUV,45,39,729,414,1,AM General Hummer SUV 2000,800,600,480000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01117.jpg
2,SUV,26,62,763,504,1,AM General Hummer SUV 2000,800,520,416000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01518.jpg
3,SUV,32,69,487,316,1,AM General Hummer SUV 2000,500,353,176500,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01538.jpg
4,SUV,46,115,623,469,1,AM General Hummer SUV 2000,786,491,385926,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01802.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8436,Convertible,20,27,613,413,196,smart fortwo Convertible 2012,630,420,264600,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07500.jpg
8437,Convertible,29,34,381,273,196,smart fortwo Convertible 2012,400,300,120000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07594.jpg
8438,Convertible,78,289,669,633,196,smart fortwo Convertible 2012,1024,683,699392,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07846.jpg
8439,Convertible,31,6,494,272,196,smart fortwo Convertible 2012,500,272,136000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07895.jpg


## Limit to 256x256 and above

In [9]:
selected_cars_annos_resizable = selected_cars_annos.query('width >= 256 and height >= 256')
selected_cars_annos_resizable

,Class,x1,y1,x2,y2,old_class,old_class_name,width,height,num_pixels,test_80_20,new_file_path,new_file_name
1,SUV,45,39,729,414,1,AM General Hummer SUV 2000,800,600,480000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01117.jpg
2,SUV,26,62,763,504,1,AM General Hummer SUV 2000,800,520,416000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01518.jpg
3,SUV,32,69,487,316,1,AM General Hummer SUV 2000,500,353,176500,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01538.jpg
4,SUV,46,115,623,469,1,AM General Hummer SUV 2000,786,491,385926,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01802.jpg
5,SUV,11,60,796,535,1,AM General Hummer SUV 2000,800,580,464000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01887.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8436,Convertible,20,27,613,413,196,smart fortwo Convertible 2012,630,420,264600,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07500.jpg
8437,Convertible,29,34,381,273,196,smart fortwo Convertible 2012,400,300,120000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07594.jpg
8438,Convertible,78,289,669,633,196,smart fortwo Convertible 2012,1024,683,699392,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07846.jpg
8439,Convertible,31,6,494,272,196,smart fortwo Convertible 2012,500,272,136000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07895.jpg


## Load Image, Resize, and Save

In [5]:
# Rename new_file_path to source_file_path
selected_cars_annos_resizable = selected_cars_annos_resizable.rename(columns={'new_file_path': 'source_file_path'})
# Create destination_file_name as new_file_name with the last ".jpg" replaced with "_resized.jpg"
selected_cars_annos_resizable['destination_file_name'] = selected_cars_annos_resizable['new_file_name'].apply(lambda x: x[:x.rfind('.jpg')] + '_resized.jpg')
# Print source_file_path and destination_file_path for first row
print(os.path.expanduser(selected_cars_annos_resizable.iloc[0]['source_file_path']))
print(selected_cars_annos_resizable.iloc[0]['destination_file_name'])
# Create destination_file_path as source_file_path with "Selected Images" replaced with "Resized Images" and with the last ".jpg" replaced with "_resized.jpg"
selected_cars_annos_resizable['destination_file_path'] = selected_cars_annos_resizable['source_file_path'].apply(lambda x: x.replace('Selected Images', 'Resized Images')).apply(lambda x: x[:x.rfind('.jpg')] + '_resized.jpg')
print(os.path.expanduser(selected_cars_annos_resizable.iloc[0]['destination_file_path']))

C:\Users\ijyli/Box/INFO 290T Project/Intermediate Data/Selected Images/train/SUV_train_01117.jpg
SUV_train_01117_resized.jpg
C:\Users\ijyli/Box/INFO 290T Project/Intermediate Data/Resized Images/train/SUV_train_01117_resized.jpg


### Testing resizing on 10 images

In [5]:
# Draw first 10 images
resizing_sample = selected_cars_annos_resizable.head(10)

# Change destination_file_path "~/Box/INFO 290T Project/Intermediate Data/Small Resized Sample/" + destination_file_name
resizing_sample['destination_file_path'] = "~/Box/INFO 290T Project/Intermediate Data/Small Resized Sample/" + resizing_sample['destination_file_name']

# Clear sample directory
shutil.rmtree(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Small Resized Sample/"), ignore_errors=True)
os.makedirs(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Small Resized Sample/"), exist_ok=True)

# Timing run
sample_start_time = time.time()

# Iterate through the sample and process each image
for _, row in resizing_sample.iterrows():
    process_image(row['source_file_path'], row['destination_file_path'])

# End timing run
sample_end_time = time.time()

# Sample time taken in minutes
sample_time_taken = (sample_end_time - sample_start_time) / 60
print('Sample time taken: ', sample_time_taken, ' minutes')

# Estimate time for full run
full_time_est = sample_time_taken * len(selected_cars_annos_resizable) / 10
print('Full run time estimated: ', full_time_est, ' minutes')

C:\Users\ijyli\AppData\Local\Temp\ipykernel_6732\719799075.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resizing_sample['destination_file_path'] = "~/Box/INFO 290T Project/Intermediate Data/Small Resized Sample/" + resizing_sample['destination_file_name']


Sample time taken:  0.024089503288269042  minutes
Full run time estimated:  19.878658113479613  minutes


### Run on All Images

In [6]:
# Clear full directory
shutil.rmtree(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Resized Images/"), ignore_errors=True)
# Make subdirectories
os.makedirs(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Resized Images/train"), exist_ok=True)
os.makedirs(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Resized Images/test"), exist_ok=True)

In [7]:
# Start timer
full_start_time = time.time()

# Iterate through the full set and process each image
for _, row in selected_cars_annos_resizable.iterrows():
    process_image(row['source_file_path'], row['destination_file_path'])

# End timer
full_end_time = time.time()

# Full time taken in minutes
print('Full time taken: ', (full_end_time - full_start_time) / 60, ' minutes')

Full time taken:  31.189873480796813  minutes


In [8]:
# Check success of operation - number of files in the source and destination directories
print('number of files to resize: ', len(selected_cars_annos_resizable))
print('number of resized files: ', len(os.listdir(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Resized Images/train"))) + len(os.listdir(os.path.expanduser("~/Box/INFO 290T Project/Intermediate Data/Resized Images/test"))))
print('items above 260 w and h: ', len(selected_cars_annos_resizable.query('width > 260 and height > 260')))

number of files to resize:  8252
number of resized files:  128
items above 260 w and h:  8232


## Modify and Save Excel File

In [9]:
selected_cars_annos_resizable

,Class,x1,y1,x2,y2,old_class,old_class_name,width,height,num_pixels,test_80_20,source_file_path,new_file_name,destination_file_name,destination_file_path
1,SUV,45,39,729,414,1,AM General Hummer SUV 2000,800,600,480000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01117.jpg,SUV_train_01117_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
2,SUV,26,62,763,504,1,AM General Hummer SUV 2000,800,520,416000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01518.jpg,SUV_train_01518_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
3,SUV,32,69,487,316,1,AM General Hummer SUV 2000,500,353,176500,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01538.jpg,SUV_train_01538_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
4,SUV,46,115,623,469,1,AM General Hummer SUV 2000,786,491,385926,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01802.jpg,SUV_train_01802_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
5,SUV,11,60,796,535,1,AM General Hummer SUV 2000,800,580,464000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,SUV_train_01887.jpg,SUV_train_01887_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8436,Convertible,20,27,613,413,196,smart fortwo Convertible 2012,630,420,264600,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07500.jpg,Convertible_train_07500_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
8437,Convertible,29,34,381,273,196,smart fortwo Convertible 2012,400,300,120000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07594.jpg,Convertible_train_07594_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
8438,Convertible,78,289,669,633,196,smart fortwo Convertible 2012,1024,683,699392,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07846.jpg,Convertible_train_07846_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...
8439,Convertible,31,6,494,272,196,smart fortwo Convertible 2012,500,272,136000,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,Convertible_train_07895.jpg,Convertible_train_07895_resized.jpg,~/Box/INFO 290T Project/Intermediate Data/Resi...


In [10]:
# Create df resized_cars_annos
# Keep columns Class, old_class_name, test_80_20, source_file_path renamed to orig_res_file_path, destination_file_path renamed to resized_file_path
resized_cars_annos = selected_cars_annos_resizable[['Class', 'old_class_name', 'width', 'height', 'test_80_20', 'source_file_path', 'destination_file_path']]
resized_cars_annos = resized_cars_annos.rename(columns={'source_file_path': 'orig_res_file_path', 'destination_file_path': 'resized_file_path', 'width': 'orig_width', 'height': 'orig_height'})
resized_cars_annos

,Class,old_class_name,orig_width,orig_height,test_80_20,orig_res_file_path,resized_file_path
1,SUV,AM General Hummer SUV 2000,800,600,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
2,SUV,AM General Hummer SUV 2000,800,520,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
3,SUV,AM General Hummer SUV 2000,500,353,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
4,SUV,AM General Hummer SUV 2000,786,491,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
5,SUV,AM General Hummer SUV 2000,800,580,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
...,...,...,...,...,...,...,...
8436,Convertible,smart fortwo Convertible 2012,630,420,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
8437,Convertible,smart fortwo Convertible 2012,400,300,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
8438,Convertible,smart fortwo Convertible 2012,1024,683,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...
8439,Convertible,smart fortwo Convertible 2012,500,272,0,~/Box/INFO 290T Project/Intermediate Data/Sele...,~/Box/INFO 290T Project/Intermediate Data/Resi...


In [11]:
# Save resized_cars_annos to ~/Box/INFO 290T Project/Intermediate Data/resized_cars_annos.xlsx
resized_cars_annos.to_excel("~/Box/INFO 290T Project/Intermediate Data/resized_cars_annos.xlsx", index=False)